<a href="https://colab.research.google.com/github/shuen1106lu/Program-Language/blob/main/%E7%AC%AC%E5%85%AB%E5%91%A8_%E6%96%87%E5%AD%97%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**關於「幻想鄉」的描述與簡介，及常見字詞的特色**

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1D072ZNIrEiR3FKym76eQ_R0ywGVsGh6EddzmnMnVV5M/edit?usp=sharing').sheet1

In [3]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1D072ZNIrEiR3FKym76eQ_R0ywGVsGh6EddzmnMnVV5M/edit?usp=sharing').get_worksheet(1)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [4]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [5]:
df.head()

,Sections,Cleaned_Content
0,前言,「幻想鄉」是日本同人遊戲系列《東方Project》（由ZUN／太田順也創作）的核心舞台，是一...
1,地理位置與結界,幻想鄉原本位於日本本州東北部的深山（現實中的岐阜縣白川鄉附近），但隨著外界科學文明的發展，1...
2,社會結構,人類與非人類共存：人類村落（主要位於「人間之里」）與妖怪、神明、妖精等超自然生物形成微妙的平...
3,人間之里,普通人類居住地，受慧音（半獸白澤）的歷史知識保護，避免被妖怪侵擾。村民對妖怪既恐懼又依賴（如...
4,妖怪之山,天狗（新聞社《文々。新聞》）、河童（科技研發）、守矢神社（外來神明風神諏訪子與八坂神奈子）的...


In [6]:
df.describe()

,Sections,Cleaned_Content
count,30,30
unique,30,30
top,前言,「幻想鄉」是日本同人遊戲系列《東方Project》（由ZUN／太田順也創作）的核心舞台，是一...
freq,1,1


In [7]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=69d08550dbf67348f0ca05e580023eef9ebff47780ac5432a396748e7f7dbb3b
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [8]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [9]:
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.845 seconds.
DEBUG:jieba:Loading model cost 0.845 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [10]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [11]:
df['Chinese_Content'] = df['Cleaned_Content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [12]:
df['Chinese_Content']

,Chinese_Content
0,幻想鄉是日本同人遊戲系列東方由太田順也創作的核心舞台是一個充滿妖怪神明人類與超自然現象的封閉...
1,幻想鄉原本位於日本本州東北部的深山現實中的岐阜縣白川鄉附近但隨著外界科學文明的發展世紀末年被...
2,人類與非人類共存人類村落主要位於人間之里與妖怪神明妖精等超自然生物形成微妙的平衡妖怪需要人類...
3,普通人類居住地受慧音半獸白澤的歷史知識保護避免被妖怪侵擾村民對妖怪既恐懼又依賴如依賴巫女退治...
4,天狗新聞社文新聞河童科技研發守矢神社外來神明風神諏訪子與八坂神奈子的據點展現科技與信仰的衝突...
5,吸血鬼蕾米莉亞斯卡蕾特的洋館象徵西洋幻想的入侵與日式妖怪形成對比女僕長十六夜咲夜的能力操控時...
6,生與死的界限模糊幽幽子亡靈管理者與映姬地獄審判體現東方對輪迴與罪罰的詮釋
7,幻想鄉是被遺忘之物的避難所外界失去信仰的神明如守矢雙神與瀕危妖怪在此延續隱喻傳統文化在現代社...
8,無絕對善惡妖怪吃人是天性人類退治妖怪是自保雙方達成動態平衡如東方儚月抄中月之民與地上生物的衝...
9,表面是烏托邦自由奇幻實則隱藏危險弱肉強食異變頻發通過異變劇情如東方紅魔鄉的紅霧揭露幻想鄉的脆弱性


In [13]:
df['Tokenized']

,Tokenized
0,"[幻想, 鄉, 是, 日本, 同人, 遊戲, 系列, 東方, 由, 太田, 順, 也, 創作..."
1,"[幻想, 鄉, 原本, 位, 於, 日本, 本州, 東, 北部, 的, 深山, 現實, 中,..."
2,"[人類, 與, 非人, 類, 共存, 人類, 村落, 主要, 位, 於, 人間之里, 與, ..."
3,"[普通人, 類, 居住地, 受慧音, 半獸, 白澤, 的, 歷史, 知識, 保護, 避免, ..."
4,"[天狗, 新聞, 社文新聞, 河童, 科技, 研發守, 矢, 神社, 外來, 神明, 風神,..."
5,"[吸血鬼, 蕾, 米莉亞斯卡蕾特, 的, 洋館, 象, 徵, 西洋, 幻想, 的, 入侵, ..."
6,"[生, 與, 死, 的, 界限, 模糊, 幽幽, 子亡靈, 管理者, 與, 映姬, 地獄, ..."
7,"[幻想, 鄉, 是, 被, 遺忘之物, 的, 避難, 所, 外界, 失去, 信仰, 的, 神..."
8,"[無絕, 對, 善惡, 妖怪, 吃, 人, 是, 天性, 人類, 退治, 妖怪, 是, 自保..."
9,"[表面, 是, 烏托邦, 自由, 奇幻, 實則, 隱藏, 危險弱, 肉強, 食異, 變頻, ..."


In [14]:
word_counts = Counter(all_words)
word_counts

Counter({'幻想': 30,
         '日本': 6,
         '同人': 3,
         '遊戲': 4,
         '系列': 2,
         '東方': 6,
         '太田': 1,
         '創作': 3,
         '核心': 2,
         '舞台': 2,
         '一個': 4,
         '充滿': 1,
         '妖怪': 24,
         '神明': 7,
         '人類': 12,
         '超自然': 2,
         '現象': 3,
         '閉異': 1,
         '世界': 9,
         '設定': 10,
         '融合': 2,
         '傳統': 5,
         '民俗': 3,
         '道教': 1,
         '文化': 9,
         '獨創': 1,
         '元素': 2,
         '形成': 4,
         '懷舊': 2,
         '創新': 2,
         '獨特': 1,
         '以下': 1,
         '從多個': 1,
         '角度': 1,
         '詳細': 1,
         '分析': 1,
         '評價': 1,
         '原本': 1,
         '本州': 1,
         '北部': 1,
         '深山': 1,
         '現實': 4,
         '岐阜': 1,
         '縣白川鄉': 1,
         '附近': 1,
         '隨著': 1,
         '外界': 14,
         '科學': 3,
         '文明': 2,
         '發展': 2,
         '世紀': 1,
         '末年': 1,
         '博麗大': 2,
         '結界': 6,
         '離成': 1,


In [19]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以','存在','可能','反映','依賴','這種','一個','例如','導致', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [20]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [21]:
word_freq_df

,Word,Frequency
0,幻想,30
1,妖怪,24
2,外界,14
3,人類,12
4,設定,10
...,...,...
590,記憶,1
591,折衷,1
592,和洋,1
593,極端化,1


In [22]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 3s (2,846 kB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 126315 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [23]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [25]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [ ]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請分析以上的回答，猜測這描述的是一個怎樣的世界觀，並判斷為何一系列官方/非官方作品能吸引大量玩家/非玩家關注並喜愛：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)